In [1]:
import anvil.server
import anvil.media
from anvil import BlobMedia
anvil.server.connect("4LWTNNDYE2AJJJCWPGPFAFHX-CLFFCZ4AB5DTBE3B")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import cv2 
import pytesseract as pytesseract
#pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\JCheng10\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
import tensorflow as tf
from PIL import Image
import sys
from pdf2image import convert_from_path
from PyPDF2 import PdfFileReader, PdfFileWriter
import os
import cv2
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
from keras import backend as K
#K.set_image_dim_ordering('tf')
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
#from keras.optimizers import RMSprop
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
#categories

classes = ['Drift report', 'NDE report', 'Threading Cert', 'Work order', 'Traceability log sheet',
          'Hardness report', 'Material Certs', 'Certificate of conformance', 'Diagrams',
          'Pressure test', 'Delivery invoice']

category_codes = {
    'Drift report': 0,
    'Hardness report': 1,
    'Material Certs': 2,
    'NDE report': 3,
    'Threading Cert': 4,
    'Work order': 5,
    'Traceability log sheet': 6,
    'Pressure test': 7,
    'Diagrams': 8,
    'Certificate of conformance': 9,
    'Delivery invoice':10
}

#base = r'C:\Users\JCheng10\OneDrive - Schlumberger\Desktop\Classification'
base = str(os.path.join(Path.home(), "Desktop"))
    
newpath = base + '\\results\\'
if not os.path.exists(newpath):
    os.makedirs(newpath)

# load the model from disk
filename = 'finalized_model.sav'
model = pickle.load(open(filename, 'rb'))
file = 'vectorizer.sav'
vectorizer = pickle.load(open(file, 'rb'))
file2 = 'vectorizer_gib.sav'
vectorizer_gib = pickle.load(open(file2, 'rb'))
file3 = 'gibberish.sav'
gibberish = pickle.load(open(file3, 'rb'))

In [4]:
category_names = {
    0:'Drift report',
    1:'Hardness report',
    2:'Material Certs',
    3:'NDE report',
    4:'Threading Cert',
    5:'Work order',
    6:'Traceability log sheet',
    7:'Pressure test',
    8:'Diagrams',
    9:'Certificate of conformance',
    10:'Delivery invoice'
}

In [15]:
def create_csv(path):

    files = os.listdir(path)
    
    img_list = []
    
    for filename in files:
        if filename.lower().endswith(('.jpg', '.jpeg')):
            img_list.append(filename)
    
    df = pd.DataFrame()
   
    df['images']=img_list
        
    return df

def convert_to_image(path, name):
    newpath = base + '\\results\\'
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    pages = convert_from_path(path, 500)
    image_counter=1
    for page in pages:
        filename = name[:-4] + '_' + str(image_counter)+".jpg"

        # Save the image of the page in system
        page.save(newpath + filename, 'JPEG')

        # Increment the counter to update filename
        image_counter += 1
    return pages

def preprocess(image):
    image = cv2.medianBlur(image,5)

    kernel = np.ones((5,5),np.uint8)
    image = cv2.dilate(image, kernel, iterations = 1)
    
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

def extract_text(df):
    df['text'] = ''
    for i in range(len(df)):
        # Creating a text file to write the output
        #outfile = df['document'][i][:-4] + ".txt"

        # Open the file in append mode so that 
        # All contents of all images are added to the same file
        #f = open(outfile, "w")
        #custom_config = r'--oem 3 --psm 6' #read normally line by line
        img = cv2.imread(newpath + df['images'][i])
        img = preprocess(img)
        custom_config = r'-c tessedit_char_blacklist=0123456789 --psm 6' #dont read numbers
        text = str(((pytesseract.image_to_string(img, config=custom_config))))

        #f.write(text)

        # Close the file after writing all the text.
        #f.close()
        textf = check_text(text, img)
        df['text'][i] = textf

def data_cleaning(data):
    #cleaning
    #\r and \n
    data['text_1'] = data['text'].str.replace("\r", " ")
    data['text_1'] = data['text_1'].str.replace("\n", " ")
    data['text_1'] = data['text_1'].str.replace("    ", " ")

    # " when quoting text
    data['text_1'] = data['text_1'].str.replace('"', '')

    # Lowercasing the text
    data['text_1'] = data['text_1'].str.lower()

    punctuation_signs = list("?:!.,;")

    for punct_sign in punctuation_signs:
        data['text_1'] = data['text_1'].str.replace(punct_sign, '')

    data['text_1'] = data['text_1'].str.replace("'s", "")

def put_into_folders(df):
    df['Category_Predicted'] = df['predictions']
    df = df.replace({'Category_Predicted':category_names})
    for i in range(len(df)):
        Path(newpath+df['images'][i]).rename(newpath+df['Category_Predicted'][i]+'\\'+df['images'][i])

def check_text(text, img):
    custom_config = r'-c tessedit_char_blacklist=0123456789 --psm 6'
    text_tfm = [text]
    if gibberish.predict(vectorizer_gib.transform(text_tfm)) == 1:
        return text
    else:
        img1 = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        text1 = str(((pytesseract.image_to_string(img1, config=custom_config))))
        text_tfm = [text1]
        if gibberish.predict(vectorizer_gib.transform(text_tfm)) == 1:
            return text1
        else: 
            img2 = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            text2 = str(((pytesseract.image_to_string(img2, config=custom_config))))
            return text2

def convert_combine(name):
    for i in os.listdir(newpath):
        indiv = newpath + i
        if len(os.listdir(indiv)) > 1:
            imglist = []
            #numlist = []
            for j in os.listdir(indiv):
                if j.lower().endswith(('.jpg', '.jpeg')):
                    im = Image.open(indiv+'\\'+j)
                    im.convert('RGB')
                    imglist.append(im)
                
            im1 = imglist.pop(0)
            newname = indiv+'\\'+name+'.pdf'
            im1.save(newname, "PDF" ,resolution=500, save_all=True, append_images=imglist)
        elif len(os.listdir(indiv)) == 1:
            im = Image.open(indiv+'\\'+ os.listdir(indiv)[0])
            im.convert('RGB')
            newname = indiv+'\\'+name #+'.pdf'
            im.save(newname, "PDF" ,resolution=500)
            
        for file in os.listdir(indiv):
            if file.endswith(('.jpg', '.jpeg')):
                os.remove(indiv+'\\' + file)

In [16]:
@anvil.server.callable

def classify(file, filename): 
    with anvil.media.TempFile(file) as filepath:
        pages = convert_to_image(filepath, filename) 
    print("Images formed!")
    
    df = pd.DataFrame()
    df = create_csv(newpath)
    print("Dataframe created!")
    
    
    extract_text(df)
    print("Text extracted!")
    
    #return df
    
    
    data_cleaning(df)
    print("Text cleaned!")
    

    model_input = vectorizer.transform(df.text_1)
    
    preds = model.predict(model_input)
    print("Model succeeded!")
    
    
    df['predictions'] = preds
    print("Dataframe updated!")
    
    #return df
    
    #making individual folders after splitting and categorising
  
    for i in classes:
        folders = newpath + i
        if not os.path.exists(folders):
            os.makedirs(folders)

    put_into_folders(df)
    print("All files split into their respective folders!")
    
    convert_combine(filename)
    print("Converted and combined too!")

    return True

#anvil.server.wait_forever()

Images formed!
Dataframe created!
Text extracted!
Text cleaned!
Model succeeded!
Dataframe updated!
All files split into their respective folders!


<ipython-input-15-26116488d735>:80: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  data['text_1'] = data['text_1'].str.replace(punct_sign, '')


Converted and combined too!
Images formed!
Dataframe created!
Text extracted!
Text cleaned!
Model succeeded!
Dataframe updated!
All files split into their respective folders!
Converted and combined too!
